In [7]:
import pickle

# Specify the path to the .pkl file
file_path = '/home/nick/BERT4Rec_modified/Data/preprocessed/ml-1m_min_rating0-min_uc5-min_sc0-splitleave_one_out/dataset.pkl'

# Open the file in binary mode
with open(file_path, 'rb') as file:
    # Load the data from the file
    data = pickle.load(file)



In [89]:
data['train'][0]


[2970,
 1575,
 958,
 1179,
 2148,
 1659,
 3178,
 1118,
 2600,
 690,
 1105,
 254,
 859,
 594,
 2489,
 1782,
 1849,
 2890,
 878,
 1783,
 971,
 145,
 1839,
 964,
 1026,
 854,
 2593,
 1196,
 2558,
 1155,
 640,
 2711,
 518,
 2899,
 2587,
 965,
 2129,
 1108,
 581,
 2206,
 1422,
 514,
 1,
 575,
 2484,
 709,
 582,
 2103,
 741,
 2163,
 1728]

In [90]:
import torch

# 创建一个示例输入序列 x
x = torch.tensor([[1, 2, 3, 0],
                  [4, 5, 0, 0]])
print(x.shape)
# 生成掩码
# mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)

print("输入序列 x：\n", x)
# print("生成的掩码 mask：\n", mask)
print("x>0: ", x>0)
print("(x > 0).unsqueeze(1): ", (x > 0).unsqueeze(1))
print("(x > 0).unsqueeze(1).repeat(1, x.size(1), 1): ", (x > 0).unsqueeze(1).repeat(1, x.size(1), 1))
print("mask: ", (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1))

torch.Size([2, 4])
输入序列 x：
 tensor([[1, 2, 3, 0],
        [4, 5, 0, 0]])
x>0:  tensor([[ True,  True,  True, False],
        [ True,  True, False, False]])
(x > 0).unsqueeze(1):  tensor([[[ True,  True,  True, False]],

        [[ True,  True, False, False]]])
(x > 0).unsqueeze(1).repeat(1, x.size(1), 1):  tensor([[[ True,  True,  True, False],
         [ True,  True,  True, False],
         [ True,  True,  True, False],
         [ True,  True,  True, False]],

        [[ True,  True, False, False],
         [ True,  True, False, False],
         [ True,  True, False, False],
         [ True,  True, False, False]]])
mask:  tensor([[[[ True,  True,  True, False],
          [ True,  True,  True, False],
          [ True,  True,  True, False],
          [ True,  True,  True, False]]],


        [[[ True,  True, False, False],
          [ True,  True, False, False],
          [ True,  True, False, False],
          [ True,  True, False, False]]]])


In [102]:
x = torch.tensor([[2,3], [5,4]])
print(x)
print(x.transpose(-1, -2))

tensor([[2, 3],
        [5, 4]])
tensor([[2, 5],
        [3, 4]])


In [13]:
import torch
import torch.nn as nn

# 假设有一个批次包含3个样本，每个样本的预测结果是模型的输出
predictions = torch.tensor([0.7, 0.3, 0.5], requires_grad=True)
# 相应的真实标签
labels = torch.tensor([1.0, 0.0, 1.0])

# 创建BCELoss对象
criterion = nn.BCELoss()

# 计算损失
loss = criterion(predictions, labels)

# 打印损失
print("Loss:", loss)


Loss: tensor(0.4688, grad_fn=<BinaryCrossEntropyBackward>)


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

criterion = nn.CrossEntropyLoss()

x = torch.Tensor([[1,2,3]])
y = torch.Tensor([2]).type(torch.long)

t = 1
out = F.softmax(x/t, dim=1)
print(f'After Softmax: {out}')


loss = criterion(out,y)
print(f'Loss: {loss}')


After Softmax: tensor([[0.0900, 0.2447, 0.6652]])
Loss: 0.7971903681755066


In [31]:
from torch import Tensor
import torch
import torch.nn as nn
import torchvision.models as models
torch.manual_seed(40)

class MILAttention(nn.Module):
    def __init__(self, featureLength = 512, featureInside = 256):
        '''
        Parameters:
            featureLength: Length of feature passed in from feature extractor(encoder)
            featureInside: Length of feature in MIL linear
        Output: tensor
            weight of the features
        '''
        super(MILAttention, self).__init__()
        self.featureLength = featureLength
        self.featureInside = featureInside

        self.attetion_V = nn.Sequential(  ## Linear
            nn.Linear(self.featureLength, self.featureInside, bias = True),
            nn.Tanh()
        )
        self.attetion_U = nn.Sequential(  ## Linear
            nn.Linear(self.featureLength, self.featureInside, bias = True),
            nn.Sigmoid()
        )
        self.attetion_weights = nn.Linear(self.featureInside, 1, bias = True)
        self.softmax = nn.Softmax(dim = 1)
        
    def forward(self, x: Tensor) -> Tensor:
        bz, pz, fz = x.shape
        x = x.view(bz*pz, fz)
        att_v = self.attetion_V(x)
        att_u = self.attetion_U(x)
        att = self.attetion_weights(att_u*att_v) # multiply them, and Linear
        weight = att.view(bz, pz, 1)
        
        weight = self.softmax(weight) # each batch softmax
        weight = weight.view(bz, 1, pz)

        return weight

In [42]:
x = torch.tensor([[[1.0, 2.0, 3.0, 4.0],
                   [2.0, 3.0, 4.0, 5.0],
                   [3.0, 4.0, 5.0, 6.0]],
                  
                  [[4.0, 5.0, 6.0, 7.0],
                   [5.0, 6.0, 7.0, 8.0],
                   [6.0, 7.0, 8.0, 9.0]]])  # 形状为 (2, 3, 4)

bz, pz, fz = x.shape
print(x.shape)

attention_layer = MILAttention(4, 2)
weight = attention_layer(x)
weight_sum = torch.bmm(weight, x)
weight_sum_sq = weight_sum.squeeze(1)

print(weight)
print(x)
print(weight_sum)
print(weight_sum_sq)

torch.Size([2, 3, 4])
tensor([[[0.2955, 0.3370, 0.3675]],

        [[0.3226, 0.3345, 0.3429]]], grad_fn=<ViewBackward>)
tensor([[[1., 2., 3., 4.],
         [2., 3., 4., 5.],
         [3., 4., 5., 6.]],

        [[4., 5., 6., 7.],
         [5., 6., 7., 8.],
         [6., 7., 8., 9.]]])
tensor([[[2.0721, 3.0721, 4.0721, 5.0721]],

        [[5.0203, 6.0203, 7.0203, 8.0203]]], grad_fn=<BmmBackward0>)
tensor([[2.0721, 3.0721, 4.0721, 5.0721],
        [5.0203, 6.0203, 7.0203, 8.0203]], grad_fn=<SqueezeBackward1>)


In [58]:
import torch

# 假设示例的数量为2，每个示例的长度为3，特征长度为4
bz, pz, featureInside = 2, 3, 4

# 创建示例的注意力权重张量 att_u 和 att_v
att_u = torch.tensor([[0.1, 0.2, 0.3, 0.4],
                      [0.5, 0.6, 0.7, 0.8],
                      [0.9, 1.0, 1.1, 1.2],
                      [1.3, 1.4, 1.5, 1.6],
                      [1.7, 1.8, 1.9, 2.0],
                      [2.1, 2.2, 2.3, 2.4]])  # 形状为 (6, 4)

att_v = torch.tensor([[0.2, 0.3, 0.4, 0.5],
                      [0.6, 0.7, 0.8, 0.9],
                      [1.0, 1.1, 1.2, 1.3],
                      [1.4, 1.5, 1.6, 1.7],
                      [1.8, 1.9, 2.0, 2.1],
                      [2.2, 2.3, 2.4, 2.5]])  # 形状为 (6, 4)

# 计算乘积 att_u * att_v
att_u_v_product = att_u * att_v
print(att_u_v_product)
att = nn.Linear(featureInside, 1, bias = True)
weight = att(att_u_v_product)
print(weight)
weight = weight.view(bz, pz, 1)
print(weight)
softmax = nn.Softmax(dim = 1)
weight = softmax(weight)
print(weight)

weight = weight.view(bz, 1, pz)

print(weight)


tensor([[0.0200, 0.0600, 0.1200, 0.2000],
        [0.3000, 0.4200, 0.5600, 0.7200],
        [0.9000, 1.1000, 1.3200, 1.5600],
        [1.8200, 2.1000, 2.4000, 2.7200],
        [3.0600, 3.4200, 3.8000, 4.2000],
        [4.6200, 5.0600, 5.5200, 6.0000]])
tensor([[-0.3822],
        [-0.3281],
        [-0.1646],
        [ 0.1083],
        [ 0.4906],
        [ 0.9823]], grad_fn=<AddmmBackward>)
tensor([[[-0.3822],
         [-0.3281],
         [-0.1646]],

        [[ 0.1083],
         [ 0.4906],
         [ 0.9823]]], grad_fn=<ViewBackward>)
tensor([[[0.3032],
         [0.3200],
         [0.3768]],

        [[0.2057],
         [0.3014],
         [0.4929]]], grad_fn=<SoftmaxBackward>)
tensor([[[0.3032, 0.3200, 0.3768]],

        [[0.2057, 0.3014, 0.4929]]], grad_fn=<ViewBackward>)


In [2]:
import pandas as pd
path = '/home/nick/BERT4Rec_modified/Data/preprocessed/ml-1m_min_rating0-min_uc5-min_sc0-splitleave_one_out/random-sample_size100-seed98765.pkl'
pd.read_pickle(path)

{0: [3572,
  1389,
  2014,
  945,
  3623,
  732,
  2058,
  1413,
  922,
  384,
  2035,
  75,
  3584,
  1501,
  3576,
  2529,
  1360,
  2155,
  2835,
  1225,
  3480,
  1855,
  170,
  426,
  3484,
  895,
  1036,
  2193,
  141,
  1078,
  1736,
  3644,
  974,
  1647,
  657,
  2960,
  3119,
  274,
  437,
  2607,
  733,
  1700,
  3689,
  871,
  3257,
  3621,
  2435,
  2808,
  3434,
  1352,
  2695,
  1582,
  2743,
  1738,
  104,
  2331,
  1821,
  3030,
  2471,
  1696,
  2125,
  2495,
  2447,
  2664,
  1866,
  2398,
  2323,
  1381,
  346,
  4,
  2570,
  947,
  1452,
  923,
  2790,
  3680,
  1252,
  493,
  134,
  1660,
  1053,
  1911,
  412,
  579,
  2560,
  3254,
  2713,
  2688,
  2774,
  468,
  593,
  2709,
  912,
  786,
  2828,
  1106,
  1741,
  2476,
  2118,
  2227],
 1: [1365,
  711,
  2187,
  1195,
  428,
  3686,
  1808,
  233,
  2761,
  322,
  2452,
  726,
  569,
  3634,
  2178,
  1152,
  2692,
  1139,
  2561,
  3094,
  876,
  1709,
  1973,
  1740,
  1998,
  3683,
  2179,
  2368,
  901,
